In [39]:
# !pip install pulp
# !pip install tqdm

In [40]:
import pulp 
from pulp import *
import pandas as pd
import math
from tqdm.notebook import tqdm


In [41]:
# data와 target이 어느 동 소속인지를 나타내는 데이터 불러오기
demands = pd.read_csv('수요지 행정동 분류.csv')
facilities = pd.read_csv('target 행정동 분류.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbc in position 0: invalid start byte

In [ ]:
# target 데이터에서 바다 위에 존재하는 좌표들 배제
facilities = facilities.drop([0, 3, 4, 6, 10,18]) 

In [ ]:
demands_v = demands['행정동명'].unique()
facilities_v = facilities['행정동명'].unique()

In [ ]:
# data의 좌표들이 위치한 동네에 target의 좌표들도 존재하는 지 확인
for name in demands_v:
    if name in facilities_v:
        print('true')
    else:
        print(name)
# 중구 송월동,중구 율목동, 동구 송현1·2동, 동구 송림1동, 동구 송현3동, 동구 송림4동의 경우에는 target의 좌표가 없다

In [ ]:
# 수요 df를 동 단위로 분할
d_v_df_dict = {}
for d_v_name in demands_v:
    d_v_df_dict[d_v_name] = demands[demands['행정동명']==d_v_name]

In [ ]:
d_v_df_dict

In [ ]:
# 시설지 후보 df를 동 단위로 분할
f_v_df_dict={}
for f_v_name in facilities_v:
    f_v_df_dict[f_v_name] = facilities[facilities['행정동명']==f_v_name]

### 문제,변수 설정

In [ ]:
# i(수요지)의 개수
v_i_num_dict = {}
for key, value in d_v_df_dict.items():
    v_i_num_dict[key] = value.shape[0]
    
# j(학교 입지 후보)의 개수
v_j_num_dict = {}
for key, value in f_v_df_dict.items():
    v_j_num_dict[key] = value.shape[0]


In [ ]:
# 할당 변수 생성: a[i,j]는 요청지 i를 후보 시설지 j에 할당할지 여부를 나타냅니다.
a = LpVariable.dicts("a", ((i, j) for i in range(num_i) for j in range(num_j)), 0, 1, LpBinary)

# 시설 활성화 변수 생성: b[j]는 시설지 j의 활성화 여부를 나타냅니다.
b = LpVariable.dicts("b", (j for j in range(num_j)), 0, 1, LpBinary)

In [ ]:
print(a[0,0])

In [ ]:
# 주택 - 학교 간의 거리를 구하기 위한 함수 / 수식에서 Cur에 해당

def haversine_distance(coord1, coord2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    distance = R * c
    return distance

In [ ]:
# 각 주택에서 각 학교까지의 거리를 담은 dict 
distances = {}
for i in range(num_i):
    for j in range(num_j):
        coord_i = (demands.loc[i, 'X'], demands.loc[i, 'Y'])
        coord_j = (facilities.loc[j, 'X'], facilities.loc[j, 'Y'])
        distance = haversine_distance(coord_i, coord_j)
        distances[(i, j)] = distance


In [ ]:
# 문제 정의
model = LpProblem("Capacitated_Facility_Location_Problem", LpMinimize)

### 목적함수 설정

In [ ]:
# 좌항은 이동에 대한 비용 / 우항은 학교 설립에 대한 비용 / 단위는 만원
model += lpSum(0.3562 * distances[(i, j)] * a[(i, j)] * d[i] for i in range(num_i) for j in range(num_j)) + lpSum(b[j] * 3000000 for j in range(num_j))

### 제약식

In [ ]:
# 수요지 i는 하나의 j에만 할당되어야 함
for i in tqdm(range(num_i), desc="Constraint 1"):
    model += lpSum([a[i,j] for j in range(num_j)]) == 1
    
# 각 학교 당 배정 된 학생 수는 750 사이
for j in tqdm(range(num_j), desc="Constraint 2"):
    total_demand_for_j = pulp.lpSum([d[i] * a[i,j] for i in range(num_i)])
    model += total_demand_for_j <= 750 * b[j]
    
# 학교가 설립되었을 때만 해당 학교에 학생 배정 가능
for i in tqdm(range(num_i), desc="Constraint 3"):
    for j in range(num_j):
        model += a[i,j] <= b[j]

# 모든 학생의 통학거리는 1.5km를 넘으면 안됨
for i in tqdm(range(num_i), desc="Constraint 4"):
    for j in range(num_j):
        if distances[i,j] > 7:
            model += a[i,j] == 0


### 최적화 문제 풀기 및 결과 출력

In [ ]:
# 최적화 문제 풀기

model.solve(PULP_CBC_CMD(msg=True))

# 활성화될 시설지 확인
# activated_facilities에는 학교가 설립될 시설지의 인덱스 
activated_facilities = [j for j in range(num_j) if pulp.value(b[j]) == 1]
print(activated_facilities)

In [ ]:
# 4. 최적해가 도출이 되었는지 여부
print(f"Status: {pulp.LpStatus[prob.status]}")
# 3. 각 활성화된 시설지의 위치와 할당된 학생 수 계산
result=[]
for j in activated_facilities:
    assigned_students = sum(pulp.value(a[i, j]) for i in range(num_i))
    location = (facilities.loc[j, 'X'], facilities.loc[j, 'Y'])
    line = [location,assigned_students]
    result.append(line)
    print(f"Facility {j+1} at location {location} has {assigned_students} students assigned.")

In [ ]:
# 수요지들과 후보 시설지의 할당 내용 